In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time

In [2]:
option = Options()
option.add_argument('--window-size=1890,1030')
driver = webdriver.Chrome('./chromedriver.exe', options=option)
driver.implicitly_wait(5)

In [3]:
s = []
t = []
d = []
m = []
r = []
   
driver.get(f'https://www.edmunds.com/hyundai/ioniq-6/2023/consumer-reviews/?pagesize=50')
time.sleep(0.5)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# 별점
stars = soup.findAll('span', {'class': 'rating-stars text-primary-darker mr-0_25'})
for star in stars:
    s.append(star.text.split()[0])

# 타이틀
titles = soup.findAll('h3', {'class': 'heading-5 d-inline mb-0'})
for title in titles:
    t.append(title.text.strip('" '))

# 작성일, 차종
dates = soup.findAll('div', {'class': 'small text-gray-dark mb-2'})
for date in dates:
    date = date.text.split(',')[1]
    dateStr = '{} {}-{}'.format(date.split()[0], date.split()[3], date.split()[4])
    d.append(dateStr.split()[0])
    m.append(dateStr.split()[1])

# 리뷰
reviews = soup.findAll('div', {'class': 'truncated-text size-16'})
for review in reviews:
    r.append(review.text.split('Safety')[0])

grades = soup.findAll('div', {'justify-content-between flex-column flex-md-row row'})
gradeList = []
for grade in grades:
    gradeDict = {}
    temp = []
    gradeNames = grade.findAll('dt', {'class': 'font-weight-normal'})
    for gradeName in gradeNames:
        temp.append(gradeName.text)
    gradeStars = grade.findAll('span', {'class': 'rating-stars text-primary-darker'})
    for i in range(len(gradeStars)):
        gradeDict[temp[i]] = gradeStars[i].text[0]
    gradeList.append(gradeDict)

safety = [0 for i in range(len(grades))]
technology = [0 for i in range(len(grades))]
performance = [0 for i in range(len(grades))]
interior = [0 for i in range(len(grades))]
comfort = [0 for i in range(len(grades))]
reliability = [0 for i in range(len(grades))]
value = [0 for i in range(len(grades))]

for i in range(len(gradeList)):
    for key in gradeList[i].keys():
        if key == 'Safety':
            safety[i] = int(gradeList[i]['Safety'])
        if key == 'Technology':
            technology[i] = int(gradeList[i]['Technology'])
        if key == 'Performance':
            performance[i] = int(gradeList[i]['Performance'])
        if key == 'Interior':
            interior[i] = int(gradeList[i]['Interior'])
        if key == 'Comfort':
            comfort[i] = int(gradeList[i]['Comfort'])
        if key == 'Reliability':
            reliability[i] = int(gradeList[i]['Reliability'])
        if key == 'Value':
            value[i] = int(gradeList[i]['Value'])

df = pd.DataFrame({
    'star': s,
    'title': t,
    'date': d,
    'model': m,
    'review': r,
    'safety': safety,
    'technology': technology,
    'performance': performance,
    'interior': interior,
    'comfort': comfort,
    'reliability': reliability,
    'value': value
})
df

,star,title,date,model,review,safety,technology,performance,interior,comfort,reliability,value
0,5,Love Ioniq 6,07/27/2023,IONIQ-6,The car has a very comfortable and quiet ride....,5,5,5,5,5,5,5
1,5,700 miles review,08/01/2023,IONIQ-6,"Love the car; the sports version is zippy, the...",5,4,5,5,5,5,4
2,5,I can’t believe we went electric,09/25/2023,IONIQ-6,I continually told my wife that we did not nee...,5,5,5,5,5,5,5
3,5,Never going back to gas - Ioniq 6 is a home run,08/28/2023,IONIQ-6,"5 days in and I’m blown away with this car, it...",5,5,5,5,5,5,5
4,5,Ioniq 6 - A Very Impressive EV From Hyundai,08/26/2023,IONIQ-6,"Just for background, my last car was a Honda C...",5,5,5,4,5,5,4
5,5,Quiet luxurious ride,09/09/2023,IONIQ-6,"Ioniq 6 has the power, beauty and luxury feel ...",5,5,5,5,5,5,5
6,5,Everyone thinks I bought a Porsche!,09/27/2023,IONIQ-6,My first all-electric. Let me start out by say...,5,5,5,5,5,5,5
7,5,Great ride for a reasonable price,12/12/2023,IONIQ-6,I’ve had the car for about three weeks and lov...,5,5,5,4,5,4,5
8,4,nice car,11/15/2023,IONIQ-6,"confortable and sleek lines, my concern is tha...",0,4,4,5,4,5,5
9,5,Amazing,10/10/2023,IONIQ-6,This car is amazing. Everything works as descr...,5,5,5,5,5,5,5
